<h1 style="text-align: center;">Llama3 fine-tuning 1st attempt</h1>

# Libraries

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
colab_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', colab_path)
sys.path.insert(0,colab_path)


Mounted at /content/drive


In [ ]:
%pip install "torch==2.2.2" tensorboard

%pip install  --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"


Accessing Llama3 needs permission from Meta and needs to be requested, after receiving the permission, login to HuggingFace is required to use the access token.

In [ ]:
#Login to huggingface to access the model
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [ ]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


# Dataset

This data has been annotated manually, as a part of a sentence might be worth highlighting and not all of it, we assigned 1 to the whole sentence indicating that the sentence is worth highlighting.
The test data set is the same one used for the OPEN AI API notebook.

In [ ]:
#Import sentence dataframe
df = pd.read_csv('/content/drive/MyDrive/Final Project/Field Project/Highlighted Papers/Sentence_dataframe.csv')
df

,Sentence,File_Name,Parse_Tree,Processed_Tree,Chunks,Highlight
0,Despite highly promising characteristics of th...,cleaned_Fulltext_Highly efficient.txt,(ROOT (S (PP (IN Despite) (NP (NP (ADJP (RB hi...,(S (PP (IN Despite) (NP (NP (ADJP (RB highly) ...,Despite highly promising characteristics of th...,0
1,"Here we show that woodpile -structured Ir, con...",cleaned_Fulltext_Highly efficient.txt,(ROOT (S (ADVP (RB Here)) (NP (PRP we)) (VP (V...,(S (ADVP (RB Here)) (NP (PRP we)) (VP (VBP sho...,"Here we show that woodpile-structured Ir,consi...",1
2,"Ir nanowire building blocks, improve OER mass ...",cleaned_Fulltext_Highly efficient.txt,(ROOT (S (NP (JJ Ir) (NML (NN nanowire) (NN bu...,(S (NP (JJ Ir) (NML (NN nanowire) (NN building...,"Ir nanowire building blocks,improve OER mass ...",1
3,The WP structure secures the electrochemically...,cleaned_Fulltext_Highly efficient.txt,(ROOT (S (NP (DT The) (NNP WP) (NN structure))...,(S (NP (DT The) (NNP WP) (NN structure)) (VP (...,The structure secures the electrochemically ac...,1
4,ciency of the extended surface area of 3D WP c...,cleaned_Fulltext_Highly efficient.txt,(ROOT (NP (NP (NN ciency)) (PP (IN of) (NP (NP...,(ROOT (NP (NP (NN ciency)) (PP (IN of) (NP (NP...,ciency of the extended surface area of 3D cata...,1
...,...,...,...,...,...,...
3433,"Therefore, one can argue on the clinical impor...",cleaned_Fulltext_Sex differences in fetal grow...,"(ROOT (S (ADVP (RB Therefore)) (, ,) (NP (PRP ...","(S (ADVP (RB Therefore)) (, ,) (NP (PRP one)) ...","Therefore,one can argue on the clinical import...",0
3434,"In summary, we present fetal growth curves wit...",cleaned_Fulltext_Sex differences in fetal grow...,"(ROOT (S (PP (IN In) (NP (NN summary))) (, ,) ...","(S (PP (IN In) (NP (NN summary))) (, ,) (NP (P...","In summary,we present fetal growth curves with...",1
3435,The data covers the pregnancy period from 12 w...,cleaned_Fulltext_Sex differences in fetal grow...,(ROOT (S (S (NP (DT The) (NNS data)) (VP (VBZ ...,(S (NP (DT The) (NNS data)) (VP (VBZ covers) (...,The data covers the pregnancy period from 12 w...,1
3436,"Also, the immediate neonatal outcome demonstra...",cleaned_Fulltext_Sex differences in fetal grow...,"(ROOT (S (ADVP (RB Also)) (, ,) (NP (DT the) (...","(S (ADVP (RB Also)) (, ,) (NP (DT the) (JJ imm...","Also,the immediate neonatal outcome demonstrat...",0


In [ ]:
#Train_Val_test split
train_end_point = int(df.shape[0]*0.6)
val_end_point = int(df.shape[0]*0.8)
df_train = df.iloc[:train_end_point,:]
df_val = df.iloc[train_end_point:val_end_point,:]
df_test = df.iloc[val_end_point:,:]
print(df_train.shape, df_test.shape, df_val.shape)



(2062, 6) (688, 6) (688, 6)


In [ ]:
# Drop other columns
columns_to_drop = ['File_Name', 'Parse_Tree', 'Processed_Tree', 'Chunks']

In [ ]:
# pandas dataframe to Hugging Face Dataset
dataset_train = Dataset.from_pandas(df_train.drop(columns_to_drop,axis=1))
dataset_val = Dataset.from_pandas(df_val.drop(columns_to_drop,axis=1))
dataset_test = Dataset.from_pandas(df_test.drop(columns_to_drop,axis=1))


In [ ]:
#shuffle
dataset_train_shuffled = dataset_train.shuffle(seed=42)


In [ ]:
# Combine into a DatasetDict (HuggingFace dataset format)
dataset = DatasetDict({
    'train': dataset_train_shuffled,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Highlight'],
        num_rows: 2062
    })
    val: Dataset({
        features: ['Sentence', 'Highlight'],
        num_rows: 688
    })
    test: Dataset({
        features: ['Sentence', 'Highlight'],
        num_rows: 688
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['Sentence', 'Highlight'],
    num_rows: 2062
})

In [ ]:
#label distribution
df_train.Highlight.value_counts(normalize=True)

Highlight
0    0.561591
1    0.438409
Name: proportion, dtype: float64

In [ ]:
# DEfine class weights based on distribution
class_weights=(1/df_train.Highlight.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights


tensor([0.4384, 0.5616])

# Model Configuration

We used Llama3 8B for finetuning, as the 70B caused severe memory issues even with Google Colab's L4 GPU with High RAM.

In [ ]:
#Load model (Llama 3)
model_name = "meta-llama/Meta-Llama-3-8B" #Model we used (Other options is Meta-Llama-3-70B)

In [ ]:
#quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, #4-bit precision to reduce memory
    bnb_4bit_quant_type = 'nf4', #non-linear quantization
    bnb_4bit_use_double_quant = True, #double quantization (to imrove performance)
    bnb_4bit_compute_dtype = torch.bfloat16
)


In a nutshell, "Low-Rank Adaptation (`LoRA`) is a technique designed to refine and optimise large language models based on". [Deeper Insights](https://deeperinsights.com/ai-blog/harnessing-the-power-of-lora-in-large-language-models-a-deep-dive-into-the-future-of-ai)

In [ ]:
# LoRA
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'], # Self-attention projections
    lora_dropout = 0.05,
    bias = 'none',
    task_type = 'SEQ_CLS' #sequence classification
)

Quantization of LoRA is usually called QLoRA. Based on [TextMine](https://textmine.com/post/an-introduction-to-llm-quantization) LLM quantization is an approach to processing large amounts of language data, which converts a continuous data set to a discrete data set – ultimately minimising the number of bits needed to display the signal.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=2 #As it is a binary classification
)


In [ ]:
model = prepare_model_for_kbit_training(model)
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [ ]:
#Parameter-Efficient Fine-Tuning, applying Low-Rank Adaptation (LoRa)
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
        

In [ ]:
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id # "eos" stands for end-of-sequence
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id #match model's- and Tokenizer's pad_token_id
model.config.use_cache = False # True can cause memory issues
model.config.pretraining_tp = 1 #tensor parallelism (1 = single-device training)

## Test befor training

Test model performance before fine-tunning.

In [ ]:
sentences = df_test.Sentence.tolist()

batch_size = 8

all_outputs = []


for i in range(0, len(sentences), batch_size):
    batch_sentences = sentences[i:i + batch_size]

    # Tokenization
    inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

    #Move tensors
    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    # store logits
    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])



In [ ]:
#concatante all outputs
final_outputs = torch.cat(all_outputs, dim=0)
final_outputs

tensor([[-0.4628, -3.9900],
        [ 2.2799, -2.7185],
        [-0.7629, -2.5818],
        ...,
        [-0.0347, -1.6729],
        [-0.4159, -0.5249],
        [-0.8232, -3.1463]], device='cuda:0')

In [ ]:
#class prediction
final_outputs.argmax(axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,

In [ ]:
# Convert resulting tensor to a NumPy array on the CPU for compatibility with pandas
df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
df_test['predictions']

<ipython-input-28-7c1d2547ae19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()


2750    0
2751    0
2752    0
2753    0
2754    0
       ..
3433    1
3434    0
3435    0
3436    0
3437    0
Name: predictions, Length: 688, dtype: int64

In [ ]:
df_test['predictions'].value_counts()

predictions
0    424
1    264
Name: count, dtype: int64

In [ ]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score, f1_score

def get_performance_metrics(df_test):
    y_test = df_test.Highlight
    y_pred = df_test.predictions

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
    print("Accuracy Score:", accuracy_score(y_test, y_pred))
    print("F-1 Score:", f1_score(y_test, y_pred, average='weighted'))



In [ ]:
get_performance_metrics(df_test)

Confusion Matrix:
[[233 135]
 [191 129]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.63      0.59       368
           1       0.49      0.40      0.44       320

    accuracy                           0.53       688
   macro avg       0.52      0.52      0.52       688
weighted avg       0.52      0.53      0.52       688

Balanced Accuracy Score: 0.5181385869565217
Accuracy Score: 0.5261627906976745
F-1 Score: 0.5201963888647545


# Training and result

In [ ]:
MAX_LEN = 256 #Max length of each input

def llama_preprocessing_function(examples):
    return tokenizer(examples['Sentence'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("Highlight", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/2062 [00:00<?, ? examples/s]

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

In [ ]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer) #Padding


In [ ]:
#Metrics
from sklearn.metrics import balanced_accuracy_score, accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'balanced_accuracy': balanced_accuracy_score(labels, predictions),
        'accuracy': accuracy_score(labels, predictions),
        'f1_score': f1_score(labels, predictions, average='weighted')
    }


In [ ]:
#Custom trainer with class weights
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):

        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
#Training Arguments
training_args = TrainingArguments(
    output_dir = 'binary_highlight_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [ ]:
# Custom trainer
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

<ipython-input-35-1bea759fa04e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Balanced Accuracy,Accuracy,F1 Score
1,No log,0.747689,0.573489,0.556686,0.560310
2,0.699800,0.757258,0.596140,0.591570,0.596482


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
#Prediction function
def make_predictions(model,df_test):

  sentences = df_test.Sentence.tolist()

  batch_size = 8

  all_outputs = []

  for i in range(0, len(sentences), batch_size):
      batch_sentences = sentences[i:i + batch_size]

      # Tokenization
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

      # Move tensors
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      #store logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()


make_predictions(model,df_test)

<ipython-input-39-3a01f3775112>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()


In [ ]:
get_performance_metrics(df_test)

Confusion Matrix:
[[202 166]
 [133 187]]

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.55      0.57       368
           1       0.53      0.58      0.56       320

    accuracy                           0.57       688
   macro avg       0.57      0.57      0.57       688
weighted avg       0.57      0.57      0.57       688

Balanced Accuracy Score: 0.5666440217391304
Accuracy Score: 0.565406976744186
F-1 Score: 0.5658616690023951


We believe the F-1 score is a better metric in our case for our evaluation that represents both the precision and recall of our model.
The F-1 score increased by 4.5% after finetuning the model on the dataset.

In [ ]:
metrics = train_result.metrics
max_train_samples = len(dataset_train)
metrics["train_samples"] = min(max_train_samples, len(dataset_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        2.0
  total_flos               =  9491204GF
  train_loss               =     0.6944
  train_runtime            = 0:33:39.48
  train_samples            =       2062
  train_samples_per_second =      2.042
  train_steps_per_second   =      0.256


In [ ]:
trainer.save_model("saved_model") #save trainer

In [ ]:
!pip freeze > requirements.txt